In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import openpyxl

In [50]:
file_path = "/home/nchore/Downloads/RNINO3.4.xlsx"
xls = pd.ExcelFile(file_path)
sheet_names = xls.sheet_names 
sheet_previews = {sheet: xls.parse(sheet).head() for sheet in sheet_names}
sheet_names, sheet_previews

(['NINO3.4',
  'NINO4',
  'IODE',
  'IODW',
  'MOYALE',
  'WAJIR',
  'MANDERA',
  'LAMU',
  'MALINDI',
  'MSABAHA',
  'MTWAPA',
  'MOMBASA',
  'VOI'],
 {'NINO3.4':    YEAR       J       F       M       A     M.1     J.1     J.2     A.1  \
  0  1949  24.551  26.455  27.046  27.576  26.077  26.462  27.682  27.671   
  1  1950  25.058  26.847  27.080  27.011  26.326  26.602  27.884  28.408   
  2  1951  25.867  27.130  26.763  26.724  26.539  26.645  28.266  28.710   
  3  1952  26.277  26.809  26.328  26.752  26.410  26.184  28.394  28.081   
  4  1953  26.181  26.230  25.937  27.204  25.777  26.113  28.555  27.519   
  
          S       O       N       D  
  0  26.788  27.061  25.921  25.737  
  1  26.535  26.802  25.671  25.604  
  2  26.127  26.683  25.520  25.570  
  3  25.634  26.652  25.540  25.972  
  4  25.631  26.113  25.675  26.482  ,
  'NINO4':    year  January  February   March   April     May    June    July  August  \
  0  1949   26.943    27.219  28.112  28.553  27.711  2

Each sheet contains time-series data with sea surface temperature anomalies for a specific region

In [51]:
# Load all sheets into a dictionary with cleaned data
data_frames = {}
for sheet in sheet_names:
    df = xls.parse(sheet) 
    df = df.dropna(how="all") 
    df = df.dropna(axis=1, how="all") 
    
    df.columns = [str(col).strip() for col in df.columns]
    
    data_frames[sheet] = df

data_frames.keys(), {sheet: df.head() for sheet, df in data_frames.items()}


(dict_keys(['NINO3.4', 'NINO4', 'IODE', 'IODW', 'MOYALE', 'WAJIR', 'MANDERA', 'LAMU', 'MALINDI', 'MSABAHA', 'MTWAPA', 'MOMBASA', 'VOI']),
 {'NINO3.4':    YEAR       J       F       M       A     M.1     J.1     J.2     A.1  \
  0  1949  24.551  26.455  27.046  27.576  26.077  26.462  27.682  27.671   
  1  1950  25.058  26.847  27.080  27.011  26.326  26.602  27.884  28.408   
  2  1951  25.867  27.130  26.763  26.724  26.539  26.645  28.266  28.710   
  3  1952  26.277  26.809  26.328  26.752  26.410  26.184  28.394  28.081   
  4  1953  26.181  26.230  25.937  27.204  25.777  26.113  28.555  27.519   
  
          S       O       N       D  
  0  26.788  27.061  25.921  25.737  
  1  26.535  26.802  25.671  25.604  
  2  26.127  26.683  25.520  25.570  
  3  25.634  26.652  25.540  25.972  
  4  25.631  26.113  25.675  26.482  ,
  'NINO4':    year  January  February   March   April     May    June    July  August  \
  0  1949   26.943    27.219  28.112  28.553  27.711  28.675  28.616

There is no common column across all sheets, meaning the datasets may have different structures

In [52]:
# Check for a common date column across all sheets
common_columns = set.intersection(*(set(df.columns) for df in data_frames.values()))
common_columns


set()

In [53]:
# Display first few rows of each dataset to identify a time column
data_previews = {sheet: df.head() for sheet, df in data_frames.items()}
data_previews

{'NINO3.4':    YEAR       J       F       M       A     M.1     J.1     J.2     A.1  \
 0  1949  24.551  26.455  27.046  27.576  26.077  26.462  27.682  27.671   
 1  1950  25.058  26.847  27.080  27.011  26.326  26.602  27.884  28.408   
 2  1951  25.867  27.130  26.763  26.724  26.539  26.645  28.266  28.710   
 3  1952  26.277  26.809  26.328  26.752  26.410  26.184  28.394  28.081   
 4  1953  26.181  26.230  25.937  27.204  25.777  26.113  28.555  27.519   
 
         S       O       N       D  
 0  26.788  27.061  25.921  25.737  
 1  26.535  26.802  25.671  25.604  
 2  26.127  26.683  25.520  25.570  
 3  25.634  26.652  25.540  25.972  
 4  25.631  26.113  25.675  26.482  ,
 'NINO4':    year  January  February   March   April     May    June    July  August  \
 0  1949   26.943    27.219  28.112  28.553  27.711  28.675  28.616  28.371   
 1  1950   26.668    27.528  28.376  28.583  27.527  28.507  28.698  28.664   
 2  1951   26.523    28.004  28.127  28.568  27.555  28.190  2

Each sheet has a different structure, and only some contain a time-related column

In [54]:
# Display first few rows of each dataset to identify a time column
data_previews = {sheet: df.head() for sheet, df in data_frames.items()}
data_previews


{'NINO3.4':    YEAR       J       F       M       A     M.1     J.1     J.2     A.1  \
 0  1949  24.551  26.455  27.046  27.576  26.077  26.462  27.682  27.671   
 1  1950  25.058  26.847  27.080  27.011  26.326  26.602  27.884  28.408   
 2  1951  25.867  27.130  26.763  26.724  26.539  26.645  28.266  28.710   
 3  1952  26.277  26.809  26.328  26.752  26.410  26.184  28.394  28.081   
 4  1953  26.181  26.230  25.937  27.204  25.777  26.113  28.555  27.519   
 
         S       O       N       D  
 0  26.788  27.061  25.921  25.737  
 1  26.535  26.802  25.671  25.604  
 2  26.127  26.683  25.520  25.570  
 3  25.634  26.652  25.540  25.972  
 4  25.631  26.113  25.675  26.482  ,
 'NINO4':    year  January  February   March   April     May    June    July  August  \
 0  1949   26.943    27.219  28.112  28.553  27.711  28.675  28.616  28.371   
 1  1950   26.668    27.528  28.376  28.583  27.527  28.507  28.698  28.664   
 2  1951   26.523    28.004  28.127  28.568  27.555  28.190  2

In [55]:
time_columns = {sheet: [col for col in df.columns if "time" in col.lower() or "date" in col.lower()] for sheet, df in data_frames.items()}
time_columns

{'NINO3.4': [],
 'NINO4': [],
 'IODE': [],
 'IODW': [],
 'MOYALE': [],
 'WAJIR': [],
 'MANDERA': [],
 'LAMU': [],
 'MALINDI': [],
 'MSABAHA': [],
 'MTWAPA': [],
 'MOMBASA': [],
 'VOI': []}

In [56]:
# Checking the first column of each dataset to identify potential time columns
first_columns = {sheet: df.iloc[:, 0].head() for sheet, df in data_frames.items()}
first_columns

{'NINO3.4': 0    1949
 1    1950
 2    1951
 3    1952
 4    1953
 Name: YEAR, dtype: int64,
 'NINO4': 0    1949
 1    1950
 2    1951
 3    1952
 4    1953
 Name: year, dtype: int64,
 'IODE': 0    1949
 1    1950
 2    1951
 3    1952
 4    1953
 Name: year, dtype: int64,
 'IODW': 0    1949
 1    1950
 2    1951
 3    1952
 4    1953
 Name: year, dtype: int64,
 'MOYALE': 0    1951
 1    1952
 2    1953
 3    1954
 4    1955
 Name: 1950.0, dtype: object,
 'WAJIR': 0    1951
 1    1952
 2    1953
 3    1954
 4    1955
 Name: 1950, dtype: object,
 'MANDERA': 0    1958
 1    1959
 2    1960
 3    1961
 4    1962
 Name: 1957, dtype: object,
 'LAMU': 0    1951
 1    1952
 2    1953
 3    1954
 4    1955
 Name: 1950, dtype: object,
 'MALINDI': 0    1962
 1    1963
 2    1964
 3    1965
 4    1966
 Name: 1961, dtype: object,
 'MSABAHA': 0    1958
 1    1959
 2    1960
 3    1961
 4    1962
 Name: 1957.0, dtype: object,
 'MTWAPA': 0    1960
 1    1961
 2    1962
 3    1963
 4    1964
 Name: 19

In [57]:
for sheet, df in data_frames.items():
    df.rename(columns={df.columns[0]: "Year"}, inplace=True)  
merged_data = data_frames[sheet_names[0]]
for sheet in sheet_names[1:]:
    merged_data = pd.merge(merged_data, data_frames[sheet], on="Year", how="outer")

merged_data.head()

,Year,J,F,M,A,M.1,J.1,J.2,A.1,S,O,N,D,January_x,February_x,March_x,April_x,May_x,June_x,July_x,August_x,September_x,October_x,November_x,December_x,January_y,February_y,March_y,April_y,May_y,June_y,July_y,August_y,September_y,October_y,November_y,December_y,January,February,March,April,May,June,July,August,September,October,November,December,7.7,10.7,69.6,174.8,266.3,8.1,15.7,18.8,8.0,22.4,32.6,0.0_x,...,0.5,100.5,23.8,17.5,124.5,141.1,106.7,309.2,100.5.1,467.5,179.9,219.8,85.3,93.5,0.3,49.8,122.7,243.6,129.3,43.9,32.0,44.5,12.7,197.6,126.7,16.3,27.9,0.0_y,608.8,163.8,127.3,177.8,88.4,11.4,32.8,3.3,61.2,93.1,0.6,47.7,178.3,259.1,80.7,75.4,58.8,94.0,55.6_x,226.2,80.3,3.2,36.1,67.0,55.6_y,21.4,0.0,2.3_y,4.7,24.7,1.5_y,46.7,82.0
0,1949,24.551,26.455,27.046,27.576,26.077,26.462,27.682,27.671,26.788,27.061,25.921,25.737,26.943,27.219,28.112,28.553,27.711,28.675,28.616,28.371,27.886,28.787,28.003,28.301,27.864,28.676,28.862,28.218,27.971,28.576,28.630,29.207,29.281,29.065,27.861,28.766,27.217,28.328,28.746,27.028,26.892,28.205,27.931,28.612,29.023,27.359,27.781,28.812,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1950,25.058,26.847,27.080,27.011,26.326,26.602,27.884,28.408,26.535,26.802,25.671,25.604,26.668,27.528,28.376,28.583,27.527,28.507,28.698,28.664,27.955,28.731,27.856,28.015,28.429,28.949,28.771,28.039,27.945,28.272,28.821,29.316,28.981,28.737,28.071,28.732,27.379,28.894,27.677,26.909,27.485,28.003,27.995,29.379,28.137,27.277,28.444,28.246,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1951,25.867,27.130,26.763,26.724,26.539,26.645,28.266,28.710,26.127,26.683,25.520,25.570,26.523,28.004,28.127,28.568,27.555,28.190,28.789,28.983,27.754,28.789,27.800,27.889,28.688,28.742,28.560,28.238,28.066,28.339,29.497,29.339,28.389,28.709,28.722,28.499,28.265,28.214,27.037,27.142,27.539,27.738,29.035,29.570,27.432,27.509,28.669,27.890,0.0,4.3,162.1,136.1,283.4,4.6,10.7,15.5,7.8,387.4,139.6,138.6,...,15.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.2,4.8,78.6,178.3,51.4,1.6,2.0,0.5,1.0,149.7,212.9,88.0
3,1952,26.277,26.809,26.328,26.752,26.410,26.184,28.394,28.081,25.634,26.652,25.540,25.972,26.901,28.177,27.983,28.685,27.415,28.025,28.704,29.009,27.689,28.849,27.527,27.857,28.832,28.621,28.075,28.175,28.249,28.374,29.851,28.990,27.902,28.774,28.835,28.634,29.020,27.170,26.981,27.779,27.725,28.075,29.605,28.365,27.115,27.943,28.596,28.500,0.0,15.5,6.9,226.1,62.5,7.9,7.9,6.9,31.0,56.4,160.6,36.5,...,7.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.8,24.1,25.7,126.8,8.9,0.0,0.0,2.6,6.5,38.2,19.3,158.5
4,1953,26.181,26.230,25.937,27.204,25.777,26.113,28.555,27.519,25.631,26.113,25.675,26.482,27.734,27.858,27.908,29.097,27.607,28.151,29.103,28.970,27.790,28.554,27.296,28.113,28.688,28.163,27.841,28.140,28.437,29.228,29.767,28.137,27.875,28.874,28.686,29.280,28.665,26.348,27.286,28.006,27.200,28.918,29.806,27.423,27.501,28.106,28.088,29.146,8.6,38.2,52.9,303.5,127.0,1.5,9.6,38.7,5.1,224.1,74.7,9.9,...,1.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,57.5,3.6,73.4,35.1,44.3,0.0,7.8,9.2,19.6,38.2,120.0,98.3
